In [2]:
import pandas as pd
import numpy as np
import statistics as st
import random 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
import datetime as dt
import statistics

model = RandomForestClassifier(n_estimators = 1000, random_state = 2022)

## Required Functions

In [209]:

# def fit_age(age: int) -> int:
#     if not 13 <= age <= 102:
#         return mean_age
#     return age

# def homogenize_age(age: int) -> int:
#     if not 13 <= age <= 70:
#         age = random.choices([age_values, age_proba])[0]
#     return age


# def balance_age(age):
#     if age < 0:
#         age *= -1
#     if age > 100:
#         age -= 100
#     if not 15 <= age <= 60:
#         age = mean_age
#     return age
        

# def homogenize_colour_distribution(colour):
#     if pd.isnull(colour):
#         colour = "".join(random.choices(colour_name, colour_proba))
#     return colour

def homogenize_gender(gender: str) -> str:
    if gender not in ['Male', 'Female']:
        gender = "".join(random.choices(gender.keys(), gender.values()))
    return gender

## Getting the dataset

In [3]:
df = pd.read_csv("train.csv")
df = df.rename(columns = {'Subject_Car_Colour': 'Car_Colour'})
df.head()
#df.LGA_name[df.LGA_name]
#df.head()
data = df.loc[:, ['Policy Start Date', 'Policy End Date', 'Gender', 'Age', 'First Transaction Date', 'No_Pol', 'target']]

In [4]:
df

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,ID_ZZA1SES,2010-05-25,2011-05-24,Female,30,2010-05-25,1,NaN,Black,Range Rover,Ibeju/Lekki,Ibeju-Lekki,Car Classic,1
12075,ID_ZZDAC3K,2010-10-03,2011-10-02,Female,59,2010-10-03,1,NaN,NaN,NaN,NaN,NaN,Car Classic,0
12076,ID_ZZIU2XC,2010-10-10,2011-10-08,Male,34,2010-10-10,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
12077,ID_ZZRQ1NF,2010-02-27,2011-02-26,NaN,120,2010-02-27,2,NaN,White,TOYOTA,Victoria Island,Lagos,CVTP,0


### Counting the Probability Distribution of each Colour in the Car_Colour Column

In [211]:
# Colours = dict(df.Car_Colour.value_counts())
# Coloured = df.Car_Colour.notna().sum()
# Colours = dict(map(lambda colour: (colour, Colours[colour] / Coloured), Colours.keys()))   # keys = Colours, values = Probability distribution of each car 

# colour_name = [key for key in Colours.keys()]
# colour_proba = [value for value in Colours.values()]

# def homogenize_colour_distribution(colour):
#     if pd.isnull(colour):
#         colour = "".join(random.choices(colour_name, colour_proba))
#     return colour

### Use of Probability Distribution (again) with Age Outliers

In [212]:
mean_age = int(st.mean([age for age in df['Age'] if 13 <= age <= 102]))    # The mean age between logical age values
ages = df.Age[(df['Age'] <= 70) & (df['Age'] >= 13)].value_counts().to_dict() 
ages = dict(map(lambda age: (age, ages[age] / sum(ages.values())), ages.keys()))   # probability distribution of each of the logical age values
age_values = [age for age in ages.keys()]; age_proba = [proba for proba in ages.values()]   # ages and their probabilities

def homogenize_age(age: int) -> int:   # Randomly assigns a non-logical age to a logical one based on probability distribution of the other ages
    if not 13 <= age <= 70:
        age = random.choices(list(ages.keys()), list(ages.values()))[0]
    return age

#df.Age.value_counts().to_dict()
data['Age'] = data['Age'].apply(lambda age: homogenize_age(age))
data['Age'] = data['Age'].fillna(mean_age)

### Counting the Probability Distribution of each gender in the Gender Column

In [213]:
gender = data.Gender[data['Gender'].isin(['Male', 'Female'])].value_counts().to_dict()  
print(gender)
gender = dict(map(lambda g: (g, gender[g] / sum(gender.values())), gender.keys()))
def homogenize_gender(g):
    if g not in ['Male', 'Female']:
        g = "".join(random.choices(list(gender.keys()), list(gender.values())))
    return g

def convert_gender(g: str) -> int:
    if g == 'Male':
        return 1
    elif g == 'Female':
        return 0


{'Male': 7617, 'Female': 3327}


In [214]:
data.head()
print(data.Gender)

homogenize_gender('SEX')
data['Gender'] = data['Gender'].apply(lambda gender: convert_gender(homogenize_gender(gender)))
#data.Gender

0          Male
1        Female
2          Male
3          Male
4        Entity
          ...  
12074    Female
12075    Female
12076      Male
12077       NaN
12078    Female
Name: Gender, Length: 12079, dtype: object


In [215]:
# df['Policy Start Date'] = df['Policy Start Date'].apply(lambda x: (x.split('-'))[0])
# df['Policy End Date'] = df['Policy End Date'].apply(lambda x: (x.split('-'))[0])
# df['First Transaction Date'] = df['First Transaction Date'].apply(lambda x: (x.split('-'))[0])
# df['Gender'] = df['Gender'].replace(("Male", "Female", "Entity", "SEX", "NO GENDER", "NOT STATED"), (0.05, 0.1, 0.07, 0.07, 0.05, 0.09))
# start = set(trial['Age'].to_list())
# Scc = {i: len(df[df['Subject_Car_Color'] == df.loc[i, 'Subject_Car_Color']]) for i in range(df.shape[0])}
# Scc
min_policy_start_date = data['Policy Start Date'].min()
min_policy_end_date = data['Policy End Date'].min()
min_transaction_date = data['First Transaction Date'].min()
# data['First Transaction Date']
# data['Policy Start Date'] = data['Policy Start Date'].apply(lambda x: (x.split('-'))[0])
# data['Policy End Date'] = data['Policy End Date'].apply(lambda x: (x.split('-'))[0])
# data['First Transaction Date'] = data['First Transaction Date'].apply(lambda x: (x.split('-'))[0])

data['Policy Start Date'] = pd.to_numeric(data["Policy Start Date"])
data['Policy End Date'] = pd.to_numeric(data["Policy End Date"])
data['First Transaction Date'] = pd.to_numeric(data["First Transaction Date"])

data['Policy Start Date'] = data['Policy Start Date'].apply(lambda x: x - min_policy_start_date)

data['Policy End Date'] = data['Policy End Date'].apply(lambda x: x - min_policy_end_date) 

data['First Transaction Date'] = data['First Transaction Date'].apply(lambda x: x - min_transaction_date)
# #data['Age'] = data['Age'].apply(lambda age: homogenize_age_distribution(age))
# data.head(8)

ValueError: Unable to parse string "2010-05-14" at position 0

## Calculate NaN Values in each Column

In [216]:
for col in data.columns:
    print(f"The number of NaN values in {col} is: \t {data[col].isna().sum()}")

The number of NaN values in Policy Start Date is: 	 0
The number of NaN values in Policy End Date is: 	 0
The number of NaN values in Gender is: 	 0
The number of NaN values in Age is: 	 0
The number of NaN values in First Transaction Date is: 	 0
The number of NaN values in No_Pol is: 	 0
The number of NaN values in target is: 	 0


## Data Splitting (train_test_split)

In [217]:
X_trial = data[[col for col in data.columns if col != 'target']]
y_trial = data['target']
X_trial

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol
0,2010-05-14,2011-05-13,1,30,2010-05-14,1
1,2010-11-29,2011-11-28,0,49,2010-11-29,1
2,2010-03-21,2011-03-20,1,43,2010-03-21,1
3,2010-08-21,2011-08-20,1,37,2010-08-21,1
4,2010-08-29,2010-12-31,0,20,2010-08-29,3
...,...,...,...,...,...,...
12074,2010-05-25,2011-05-24,0,30,2010-05-25,1
12075,2010-10-03,2011-10-02,0,59,2010-10-03,1
12076,2010-10-10,2011-10-08,1,34,2010-10-10,1
12077,2010-02-27,2011-02-26,1,43,2010-02-27,2


In [219]:
X_trial_train, X_trial_test, y_trial_train, y_trial_test = train_test_split(X_trial, y_trial, test_size = 0.15, random_state = 2020)

## Use Of Decision Tree Model

In [220]:
model.fit(X_trial_train, y_trial_train)

y_tr = model.predict(X_trial_train)

y_pred = model.predict(X_trial_test)

print("Classification Report : \n", classification_report(y_trial_test, y_pred))

ValueError: could not convert string to float: '2010-09-28'

In [221]:
print("For the training : \n", f1_score(y_trial_train, y_tr))

print("For the testing : \n", f1_score(y_trial_test, y_pred))

For the training : 
 0.024096385542168676


ValueError: Found input variables with inconsistent numbers of samples: [1812, 2415]

## Use of Stratified KFold Cross Validation this time

In [222]:
f1_accuracy_list = []
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2020)
for train_index, test_index in skf.split(X_trial, y_trial):
    X_train, X_test = X_trial.iloc[train_index], X_trial.iloc[test_index]
    y_train, y_test = y_trial[train_index], y_trial[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1_accuracy_list.append(f1_score(y_test, y_pred))

print('List of possible accuracy:', list(map(lambda x: 100 * x, f1_accuracy_list)))
print('\nMaximum Accuracy That can be obtained from this model is:', max(f1_accuracy_list)*100, '%')
print('\nMinimum Accuracy:', min(f1_accuracy_list)*100, '%')
print('\nOverall Accuracy:', statistics.mean(f1_accuracy_list)*100, '%')


ValueError: could not convert string to float: '2010-05-14'